## Preprocessing - 
* Adjusting the input data to ensure that there are no variables or outliers that are causing confusion in the model, such as:
  * Dropping more or fewer columns.
  * Creating more bins for rare occurrences in columns.
  * Increasing or decreasing the number of values for each bin.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
#drop rows having Status = 0
application_df = application_df.drop(application_df[application_df.STATUS == 0].index)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
#check if any N/A value
application_df.isnull().sum()

APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       1
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at AFFILIATION value counts for binning
affiliation_unique = application_df['AFFILIATION'].value_counts()
affiliation_unique

Independent         18477
CompanySponsored    15703
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [7]:
# Choose a cutoff value and create a list of afflication to be replaced


affiliation_to_replace = affiliation_unique[affiliation_unique<1000].index.tolist()
# Replace in dataframe
for app in affiliation_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18477
CompanySponsored    15703
Other                 114
Name: AFFILIATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_amt_unique = application_df['INCOME_AMT'].value_counts()
income_amt_unique

0                24385
25000-99999       3746
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        542
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Choose a cutoff value and create a list of INCOME_AMT to be replaced


income_amt_to_replace = income_amt_unique[income_amt_unique<1000].index.tolist()
# Replace in dataframe
for app in income_amt_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(app,"Other")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24385
25000-99999       3746
100000-499999     3374
Other             2789
Name: INCOME_AMT, dtype: int64

In [10]:
# Look at APPLICATION_TYPE value counts for binning
application_type_unique = application_df['APPLICATION_TYPE'].value_counts()
application_type_unique

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = application_type_unique[application_type_unique<500].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27032
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# Look at CLASSIFICATION value counts for binning
classification_unique = application_df['CLASSIFICATION'].value_counts()
classification_unique

C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_unique[classification_unique>1]


C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_unique[classification_unique<1800].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17323
C2000     6073
C1200     4837
Other     2261
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [15]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,Other,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,Other,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [16]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,1,0


In [18]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'], axis=1).values
y = application_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [19]:
X

array([[       1,     5000,        0, ...,        0,        1,        0],
       [       1,   108590,        0, ...,        1,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       ...,
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1, 36500179,        0, ...,        1,        1,        0]],
      dtype=int64)

In [20]:
y

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
number_input_features = len(X_train[0])
number_input_features

35

## Compile, Train and Evaluate the Model

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                2880      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,341
Trainable params: 5,341
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Callbacks
#from tf.keras.callbacks import Callback
mc = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_optimization_1.h5', 
                                     save_weights_only=True, save_freq=5)


In [26]:
# Compile the model and train the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc])


Train on 25720 samples
Epoch 1/100
25720/25720 [==============================] - 16s 612us/sample - loss: 0.5726 - accuracy: 0.7171
Epoch 2/100
25720/25720 [==============================] - 14s 525us/sample - loss: 0.5590 - accuracy: 0.7285
Epoch 3/100
25720/25720 [==============================] - 14s 527us/sample - loss: 0.5560 - accuracy: 0.7290
Epoch 4/100
25720/25720 [==============================] - 14s 525us/sample - loss: 0.5549 - accuracy: 0.7308
Epoch 5/100
25720/25720 [==============================] - 15s 571us/sample - loss: 0.5540 - accuracy: 0.7308
Epoch 6/100
25720/25720 [==============================] - 15s 566us/sample - loss: 0.5528 - accuracy: 0.7305
Epoch 7/100
25720/25720 [==============================] - 15s 569us/sample - loss: 0.5525 - accuracy: 0.7313
Epoch 8/100
25720/25720 [==============================] - 15s 587us/sample - loss: 0.5514 - accuracy: 0.7312
Epoch 9/100
25720/25720 [==============================] - 14s 531us/sample - loss: 0.5510 - accu

25720/25720 [==============================] - 14s 527us/sample - loss: 0.5410 - accuracy: 0.7369
Epoch 74/100
25720/25720 [==============================] - 14s 528us/sample - loss: 0.5403 - accuracy: 0.7380
Epoch 75/100
25720/25720 [==============================] - 14s 539us/sample - loss: 0.5406 - accuracy: 0.7376
Epoch 76/100
25720/25720 [==============================] - 14s 529us/sample - loss: 0.5401 - accuracy: 0.7386
Epoch 77/100
25720/25720 [==============================] - 14s 531us/sample - loss: 0.5402 - accuracy: 0.7392
Epoch 78/100
25720/25720 [==============================] - 16s 632us/sample - loss: 0.5401 - accuracy: 0.7383
Epoch 79/100
25720/25720 [==============================] - 13s 521us/sample - loss: 0.5403 - accuracy: 0.7377
Epoch 80/100
25720/25720 [==============================] - 14s 551us/sample - loss: 0.5399 - accuracy: 0.7385
Epoch 81/100
25720/25720 [==============================] - 15s 578us/sample - loss: 0.5404 - accuracy: 0.7383- loss: 0.5
Epo

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8574/8574 - 0s - loss: 0.5572 - accuracy: 0.7267
Loss: 0.5572488729300854, Accuracy: 0.7267319560050964


In [28]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_optimization_1.h5')

# Compile, Train and Evaluate the Model
* Adding more neurons to a hidden layer.


In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               3600      
_________________________________________________________________
dense_4 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 61        
Total params: 9,721
Trainable params: 9,721
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Callbacks
#from tf.keras.callbacks import Callback
mc2 = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_optimization_2.h5', 
                                     save_weights_only=True, save_freq=5)

In [31]:
# Compile the model and train the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc2])

Train on 25720 samples
Epoch 1/100
25720/25720 [==============================] - 16s 607us/sample - loss: 0.5703 - accuracy: 0.7200
Epoch 2/100
25720/25720 [==============================] - 14s 549us/sample - loss: 0.5597 - accuracy: 0.7277
Epoch 3/100
25720/25720 [==============================] - 17s 645us/sample - loss: 0.5560 - accuracy: 0.7293
Epoch 4/100
25720/25720 [==============================] - 15s 589us/sample - loss: 0.5554 - accuracy: 0.7278
Epoch 5/100
25720/25720 [==============================] - 17s 660us/sample - loss: 0.5536 - accuracy: 0.7301
Epoch 6/100
25720/25720 [==============================] - 16s 613us/sample - loss: 0.5528 - accuracy: 0.7312
Epoch 7/100
25720/25720 [==============================] - 16s 615us/sample - loss: 0.5518 - accuracy: 0.7295
Epoch 8/100
25720/25720 [==============================] - 15s 565us/sample - loss: 0.5510 - accuracy: 0.7313
Epoch 9/100
25720/25720 [==============================] - 14s 558us/sample - loss: 0.5506 - accu

25720/25720 [==============================] - 14s 530us/sample - loss: 0.5397 - accuracy: 0.7371
Epoch 74/100
25720/25720 [==============================] - 14s 532us/sample - loss: 0.5390 - accuracy: 0.7394
Epoch 75/100
25720/25720 [==============================] - 13s 511us/sample - loss: 0.5392 - accuracy: 0.7392
Epoch 76/100
25720/25720 [==============================] - 14s 558us/sample - loss: 0.5392 - accuracy: 0.7393
Epoch 77/100
25720/25720 [==============================] - 14s 561us/sample - loss: 0.5387 - accuracy: 0.7391
Epoch 78/100
25720/25720 [==============================] - 15s 596us/sample - loss: 0.5394 - accuracy: 0.7390
Epoch 79/100
25720/25720 [==============================] - 15s 577us/sample - loss: 0.5388 - accuracy: 0.7386
Epoch 80/100
25720/25720 [==============================] - 14s 559us/sample - loss: 0.5387 - accuracy: 0.7384
Epoch 81/100
25720/25720 [==============================] - 16s 629us/sample - loss: 0.5391 - accuracy: 0.7391
Epoch 82/100
2

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8574/8574 - 0s - loss: 0.5591 - accuracy: 0.7266
Loss: 0.5591464170854576, Accuracy: 0.7266153693199158


In [33]:
# Export our model to HDF5 file
nn2.save_weights('AlphabetSoupCharity_optimization_2.h5')

# Compile, Train and Evaluate the Model
* Adding more hidden layers.

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 80)                2880      
_________________________________________________________________
dense_15 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_16 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 31        
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Callbacks
#from tf.keras.callbacks import Callback
mc3 = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_optimization_3.h5', 
                                     save_weights_only=True, save_freq=5)

In [44]:
# Compile the model and train the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=75, callbacks=[mc3])

Train on 25720 samples
Epoch 1/75
25720/25720 [==============================] - 14s 535us/sample - loss: 0.5346 - accuracy: 0.7419
Epoch 2/75
25720/25720 [==============================] - 14s 556us/sample - loss: 0.5358 - accuracy: 0.7407
Epoch 3/75
25720/25720 [==============================] - 13s 522us/sample - loss: 0.5341 - accuracy: 0.7414
Epoch 4/75
25720/25720 [==============================] - 13s 505us/sample - loss: 0.5333 - accuracy: 0.7409
Epoch 5/75
25720/25720 [==============================] - 13s 513us/sample - loss: 0.5332 - accuracy: 0.7420
Epoch 6/75
25720/25720 [==============================] - 14s 530us/sample - loss: 0.5340 - accuracy: 0.7411
Epoch 7/75
25720/25720 [==============================] - 14s 539us/sample - loss: 0.5337 - accuracy: 0.7413
Epoch 8/75
25720/25720 [==============================] - 13s 510us/sample - loss: 0.5335 - accuracy: 0.7412
Epoch 9/75
25720/25720 [==============================] - 13s 517us/sample - loss: 0.5334 - accuracy: 0.7

25720/25720 [==============================] - 16s 612us/sample - loss: 0.5334 - accuracy: 0.7409
Epoch 52/75
25720/25720 [==============================] - 18s 708us/sample - loss: 0.5330 - accuracy: 0.7419
Epoch 53/75
25720/25720 [==============================] - 27s 1ms/sample - loss: 0.5330 - accuracy: 0.7411
Epoch 54/75
25720/25720 [==============================] - 22s 858us/sample - loss: 0.5335 - accuracy: 0.7418
Epoch 55/75
25720/25720 [==============================] - 15s 597us/sample - loss: 0.5331 - accuracy: 0.7418
Epoch 56/75
25720/25720 [==============================] - 15s 576us/sample - loss: 0.5334 - accuracy: 0.7416
Epoch 57/75
25720/25720 [==============================] - 15s 586us/sample - loss: 0.5329 - accuracy: 0.7419
Epoch 58/75
25720/25720 [==============================] - 17s 661us/sample - loss: 0.5331 - accuracy: 0.7411- loss: 0.5341 - accu
Epoch 59/75
25720/25720 [==============================] - 17s 645us/sample - loss: 0.5331 - accuracy: 0.7418
Epo

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8574/8574 - 0s - loss: 0.6955 - accuracy: 0.5323
Loss: 0.6955428924042222, Accuracy: 0.532306969165802


In [45]:
# Export our model to HDF5 file
nn3.save_weights('AlphabetSoupCharity_optimization_3.h5')